In [ ]:
# !pip install gcsfs

In [2]:
from google.colab import auth
import gcsfs
import numpy as np
auth.authenticate_user()


In [3]:
import json
import requests

# Opening JSON file
with open('new_train.json') as json_file:
    queries = json.load(json_file)

In [4]:
def average_precision(true_list, predicted_list, k=40):
    true_set = frozenset(true_list)
    predicted_list = predicted_list[:k]
    precisions = []
    for i,doc_id in enumerate(predicted_list):        
        if doc_id in true_set:
            prec = (len(precisions)+1) / (i+1)            
            precisions.append(prec)
    if len(precisions) == 0:
        return 0.0
    return round(sum(precisions)/len(precisions),3)

In [ ]:
import requests
from time import time
# place the domain you got from ngrok or GCP IP below. 
url = 'http://34.170.170.139:8080'

qs_res = []
for q, true_wids in queries.items():
  duration, ap = None, None
  t_start = time()
  try:
    res = requests.get(url + '/search', {'query': q})
    duration = time() - t_start
    if res.status_code == 200:
      pred_wids, _ = zip(*res.json())
      ap = average_precision(true_wids, pred_wids)
  except:
    pass
  
  qs_res.append((q, duration, ap))

In [ ]:
score = []
time = []
for i in qs_res:
  if i[2]!=None:
    score.append(i[2])
    time.append(i[1])
print(f"Average body score: {sum(score)/len(score)}")
print(f"Average body time: {sum(time)/len(time)}")
qs_res
